### 多核

* 特征融合：主要用来描述各种不同的特征融合方式，常见的方式有前期融合，就是前面所描述的将各个特征拼接在一起，后期融合本文后面会提到

* 核函数：SVM遇到线性不可分问题时，可以通过核函数将向量映射到高维空间，在高维空间线性可分

* 多核学习：在利用SVM进行训练时，会涉及核函数的选择问题，譬如线性核，rbf核等等，多核即为融合几种不同的核来训练。

两种特征融合方式的比较

    那刚才的方法有什么问题呢？

仔细想想，大致存在以下几点问题：

     1. 你所提取的所有特征，全部串在一起，一定合适么？如果我想知道哪些特征组合在一起效果很好，该怎么办？

     2. 用svm进行学习时，不同的特征最适合的核函数可能不一样，那我全部特征向量串在一起，我该如何选择核函数呢？

     3. 参数的选取。不同的特征即使使用相同的核，可能最适合的参数也不一样，那么如何解决呢？

     4. 全部特征都计算，计算时间的花销也是挺大的

对于刚才的问题，如果用前期融合，可能是用下面方式来解决：

     1. 根据经验，觉得在样本中可能表现不错的特征加进来，至于组合么，全部串在一起，或者选几个靠谱的串一起，慢慢试验，慢慢调，看哪些特征有改进就融合在一起

     2. 也是根据经验，选取普遍表现不错的RBF核，总之结果应该不会差

     3. 交叉验证是用来干嘛的？验证调优参数呗，全部特征融合在一起，再来调，尽管验证时间长，不要紧，反正模型是离线训练的，多调会也没关系。

    那是否有更好的选择方案呢？

    我们学过的SVM都是单核（single kernel）的，在使用的时候，需要我们根据经验或试验来选择用哪种核函数、怎样指定它的参数，这样很不方便。另一方面，实际应用当中，特征往往不是single domain的，而是异构的。拿图像分类来说，我们可能用到颜色相关的特征、纹理相关的特征、空间相关的特征，这几类特征对应的最佳的核函数未必相同，让他们共用同一个核函数，未必能得到最优的映射。对这些问题的思考，就引出了MKL。

简单地说，我们给定一些basekernels，比如linear,Polynomial,RBF,Sigmoid，对于每一个，可以指定多组参数，也就是一共有M个base kernels，我们想用它们的线性组合来作为最终的核函数。通过training，得到这个线性组合中每个kernel的权重d（weight）。由于融合了各种kernel，可以照顾到异构的特征；由于自动学习权重，我们就不需要费脑子想究竟用哪一个核哪一种参数，把可能的核、参数都拿过来，组合着来用就可以了。

##### 多核学习(MKL)可能是个不错的选择，该方法属于后期融合的一种，通过对不同的特征采取不同的核，对不同的参数组成多个核，然后训练每个核的权重，选出最佳核函数组合来进行分类。

多核线性组合，最经典的是simpleMKL，也被作为MKL的具体实现，应用在了计算机各领域。为了使MKL应用地更广，应对各种特征组合、各种样本量级，后来又有人提出了GMKL，G即Generalized，最优化方法用的是PGD（Projected Gradient Descend）。为了改进收敛效果，Vishwanathan又提出SPG-GMKL(Spectral Projected Gradient)，同时提出了多核的product组合。SPG-GMKL也被后来者视作state-of-art。

*  多核学习在解释性上比传统svm要强。多核学习可以明显的看到各个子核中哪些核在起作用，哪些核在一起合作效果比较好。

*  关于参数优化。曾经做过实验，关于同一特征选用同一核，但是不同参数，组合成多个核，也可以提升分类准确率。

*  多核学习相比前期特征融合在性能上会有3%~5%左右的提升。

*  通过特征选择，可以节约特征计算时间。